# 01 — Que es un Agente de IA

**Objetivo**: Construir un agente minimo desde cero usando OpenAI API pura (sin frameworks).

En este notebook implementamos el loop ReAct (Reason + Act) manualmente, demostrando que un agente no es magia: es un loop que alterna entre pensar y actuar.

## Contenido
1. Anatomia de un agente
2. Tool calling con OpenAI
3. Loop ReAct manual
4. Metricas: costo, latencia, correctness

In [ ]:
import os
import json
import time
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()
MODEL = "gpt-5-mini"

print("=" * 60)
print("SETUP")
print("=" * 60)
print(f"Modelo: {MODEL}")
print(f"API key configurada: {'Si' if os.getenv('OPENAI_API_KEY') else 'No'}")

## 1. Anatomia de un Agente

Un agente tiene 4 componentes:

```
┌─────────────────────────────────────────┐
│              AGENTE                      │
│                                          │
│  ┌──────────┐    ┌──────────────────┐   │
│  │  PLANNER │───▶│  HERRAMIENTAS    │   │
│  │  (LLM)   │◀───│  (funciones)     │   │
│  └──────────┘    └──────────────────┘   │
│       │                                  │
│       ▼                                  │
│  ┌──────────┐    ┌──────────────────┐   │
│  │ MEMORIA  │    │   GUARDRAILS     │   │
│  │ (estado) │    │   (limites)      │   │
│  └──────────┘    └──────────────────┘   │
└─────────────────────────────────────────┘
```

- **Planner (LLM)**: Decide que hacer en cada paso
- **Herramientas**: Funciones que el agente puede invocar
- **Memoria**: Estado acumulado entre pasos
- **Guardrails**: Limites de seguridad y costo

In [ ]:
# ============================================================
# HERRAMIENTAS: funciones que el agente puede invocar
# ============================================================

def calcular(expresion: str) -> str:
    """Evalua una expresion matematica simple de forma segura."""
    permitidos = set("0123456789+-*/.() ")
    if not all(c in permitidos for c in expresion):
        return "Error: caracteres no permitidos en la expresion"
    try:
        resultado = eval(expresion)  # noqa: S307 — solo caracteres numericos
        return str(resultado)
    except Exception as e:
        return f"Error: {e}"


def buscar_dato(tema: str) -> str:
    """Simula una busqueda en base de conocimiento."""
    base = {
        "python": "Python es un lenguaje interpretado creado por Guido van Rossum en 1991.",
        "machine learning": "Machine Learning es una rama de la IA que aprende patrones de datos.",
        "llm": "Un LLM (Large Language Model) es un modelo de lenguaje con miles de millones de parametros.",
        "agente": "Un agente de IA es un sistema que usa un LLM para decidir acciones de forma autonoma.",
        "rag": "RAG (Retrieval Augmented Generation) combina busqueda con generacion de texto.",
    }
    tema_lower = tema.lower()
    for clave, valor in base.items():
        if clave in tema_lower:
            return valor
    return f"No se encontro informacion sobre: {tema}"


# Registro de herramientas disponibles
TOOLS_REGISTRY = {
    "calcular": calcular,
    "buscar_dato": buscar_dato,
}

print("Herramientas registradas:")
for nombre, fn in TOOLS_REGISTRY.items():
    print(f"  - {nombre}: {fn.__doc__}")

## 2. Tool Calling con OpenAI

OpenAI permite definir herramientas como schemas JSON. El modelo decide cuando invocarlas.

In [ ]:
# ============================================================
# SCHEMAS DE HERRAMIENTAS (formato OpenAI)
# ============================================================

TOOLS_SCHEMA = [
    {
        "type": "function",
        "function": {
            "name": "calcular",
            "description": "Evalua una expresion matematica. Usa esta herramienta para cualquier calculo numerico.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expresion": {
                        "type": "string",
                        "description": "Expresion matematica a evaluar, e.g. '2 + 3 * 4'"
                    }
                },
                "required": ["expresion"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "buscar_dato",
            "description": "Busca informacion sobre un tema en la base de conocimiento.",
            "parameters": {
                "type": "object",
                "properties": {
                    "tema": {
                        "type": "string",
                        "description": "Tema a buscar, e.g. 'python', 'machine learning'"
                    }
                },
                "required": ["tema"]
            }
        }
    }
]

print(f"Schemas definidos: {len(TOOLS_SCHEMA)} herramientas")
print(json.dumps(TOOLS_SCHEMA[0]["function"], indent=2, ensure_ascii=False))

## 3. Loop ReAct Manual

El loop ReAct funciona asi:

1. El LLM recibe la pregunta y las herramientas disponibles
2. Si decide usar una herramienta, retorna un `tool_call`
3. Ejecutamos la herramienta y enviamos el resultado al LLM
4. Repetimos hasta que el LLM genere una respuesta final (sin tool calls)

In [ ]:
# ============================================================
# LOOP REACT MANUAL
# ============================================================

def react_agent(
    pregunta: str,
    tools_schema: list[dict],
    tools_registry: dict,
    model: str = MODEL,
    max_steps: int = 5,
) -> dict:
    """
    Agente ReAct minimo con OpenAI API.

    Args:
        pregunta: Pregunta del usuario.
        tools_schema: Schemas de herramientas para OpenAI.
        tools_registry: Mapa nombre -> funcion callable.
        model: Modelo a usar.
        max_steps: Maximo de pasos para evitar loops infinitos.

    Returns:
        Dict con respuesta, pasos, metricas de costo y latencia.
    """
    messages = [
        {"role": "system", "content": "Eres un asistente que responde preguntas. Usa las herramientas cuando sea necesario. Responde en espanol."},
        {"role": "user", "content": pregunta},
    ]

    pasos = []
    total_input_tokens = 0
    total_output_tokens = 0
    t_start = time.time()

    for step in range(max_steps):
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools_schema,
            tool_choice="auto",
        )

        msg = response.choices[0].message
        total_input_tokens += response.usage.prompt_tokens
        total_output_tokens += response.usage.completion_tokens

        # Si no hay tool calls, tenemos la respuesta final
        if not msg.tool_calls:
            pasos.append({"step": step + 1, "tipo": "respuesta_final", "contenido": msg.content})
            break

        # Procesar cada tool call
        messages.append(msg)
        for tool_call in msg.tool_calls:
            fn_name = tool_call.function.name
            fn_args = json.loads(tool_call.function.arguments)

            pasos.append({
                "step": step + 1,
                "tipo": "tool_call",
                "herramienta": fn_name,
                "argumentos": fn_args,
            })

            # Ejecutar la herramienta
            if fn_name in tools_registry:
                resultado = tools_registry[fn_name](**fn_args)
            else:
                resultado = f"Error: herramienta '{fn_name}' no encontrada"

            pasos[-1]["resultado"] = resultado

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": resultado,
            })

    latencia_ms = (time.time() - t_start) * 1000

    # Costo estimado (gpt-5-mini pricing)
    costo_input = total_input_tokens * 0.15 / 1_000_000
    costo_output = total_output_tokens * 0.60 / 1_000_000
    costo_total = costo_input + costo_output

    return {
        "pregunta": pregunta,
        "respuesta": pasos[-1].get("contenido", "Sin respuesta"),
        "pasos": pasos,
        "metricas": {
            "total_steps": len(pasos),
            "input_tokens": total_input_tokens,
            "output_tokens": total_output_tokens,
            "latencia_ms": round(latencia_ms, 1),
            "costo_usd": round(costo_total, 6),
        }
    }

print("Funcion react_agent definida correctamente")

In [ ]:
# ============================================================
# EJECUCION DEL AGENTE
# ============================================================

print("=" * 60)
print("PRUEBA 1: Pregunta con calculo")
print("=" * 60)

resultado1 = react_agent(
    pregunta="Cuanto es 15% de 2500? Y ademas, que es un LLM?",
    tools_schema=TOOLS_SCHEMA,
    tools_registry=TOOLS_REGISTRY,
)

print(f"\nPregunta: {resultado1['pregunta']}")
print(f"\nPasos ejecutados:")
for paso in resultado1["pasos"]:
    if paso["tipo"] == "tool_call":
        print(f"  Paso {paso['step']}: {paso['herramienta']}({paso['argumentos']}) → {paso['resultado']}")
    else:
        print(f"  Paso {paso['step']}: Respuesta final generada")

print(f"\nRespuesta: {resultado1['respuesta'][:300]}...")
print(f"\nMetricas:")
for k, v in resultado1["metricas"].items():
    print(f"  {k}: {v}")

In [ ]:
print("=" * 60)
print("PRUEBA 2: Pregunta sin herramientas necesarias")
print("=" * 60)

resultado2 = react_agent(
    pregunta="Explica brevemente que es un agente de IA.",
    tools_schema=TOOLS_SCHEMA,
    tools_registry=TOOLS_REGISTRY,
)

print(f"\nPregunta: {resultado2['pregunta']}")
print(f"Pasos: {resultado2['metricas']['total_steps']}")
print(f"Respuesta: {resultado2['respuesta'][:300]}...")
print(f"Costo: ${resultado2['metricas']['costo_usd']}")

## 4. Analisis de Resultados

Observaciones clave:
- El agente decide **autonomamente** cuando usar herramientas
- Cuando no necesita herramientas, responde directamente (1 paso)
- Cuando necesita herramientas, ejecuta el loop ReAct (2+ pasos)
- Cada paso adicional incrementa latencia y costo

### Takeaways
1. Un agente es un **loop de decision** sobre un LLM, no magia
2. Las herramientas son **funciones normales** registradas como schemas
3. El costo escala con la complejidad de la pregunta (mas pasos = mas tokens)
4. Los guardrails (`max_steps`) son esenciales para evitar loops infinitos

In [ ]:
import pandas as pd

resumen = pd.DataFrame([
    {
        "Pregunta": r["pregunta"][:40] + "...",
        "Pasos": r["metricas"]["total_steps"],
        "Tokens": r["metricas"]["input_tokens"] + r["metricas"]["output_tokens"],
        "Latencia (ms)": r["metricas"]["latencia_ms"],
        "Costo (USD)": f"${r['metricas']['costo_usd']:.6f}",
    }
    for r in [resultado1, resultado2]
])

print("=" * 60)
print("RESUMEN COMPARATIVO")
print("=" * 60)
print(resumen.to_string(index=False))